In [1]:
!pip install allennlp
!git clone https://github.com/mhagiwara/realworldnlp.git
%cd realworldnlp

     |████████████████████████████████| 7.6MB 8.4MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 163kB 45.4MB/s 
     |████████████████████████████████| 256kB 39.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 133kB 68.8MB/s 
     |████████████████████████████████| 245kB 58.7MB/s 
     |████████████████████████████████| 204kB 67.2MB/s 
     |████████████████████████████████| 1.0MB 61.2MB/s 
  Created wheel for ftfy: filename=ftfy-5.7-cp36-none-any.whl size=44593 sha256=e6b4062a23d6a529ddc73ab1485907612e6e4f3613dd181d71b9f9d4b27c7d2e
  Stored in directory: /root/.cache/pip/wheels/8e/da/59/6c8925d571aacade638a0f515960c21c0887af1bfe31908fbf
  Created wheel for jsonnet: filename=jsonnet-0.15.0-cp36-cp36m-linux_x86_64.whl size=3320540 sha256=3ef8c74d22f27f3ab8e4f923fa99b80ba31e6bbad9a45178d66dc7cd05472cd3
  Stored in directory: /root/.cache/pip/wheels/57/63/2e/da89cfe1ba08550bd7262d5d9c0

In [0]:
from typing import Dict

import numpy as np
import torch
import torch.optim as optim
from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.fields import LabelField, TextField
from allennlp.data.instance import Instance
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.training.trainer import Trainer
from overrides import overrides

from examples.sentiment.sst_classifier import LstmClassifier

In [0]:
EMBEDDING_DIM = 16
HIDDEN_DIM = 16

In [0]:
class TatoebaSentenceReader(DatasetReader):
    def __init__(self, token_indexers: Dict[str, TokenIndexer]=None, lazy=False):
        super().__init__(lazy=lazy)
        self.tokenizer = CharacterTokenizer()
        self.token_indexers = token_indexers or {'tokens': SingleIdTokenIndexer()}

    @overrides
    def text_to_instance(self, tokens, label=None):
        fields = {}

        fields['tokens'] = TextField(tokens, self.token_indexers)
        if label:
            fields['label'] = LabelField(label)

        return Instance(fields)

    @overrides
    def _read(self, file_path: str):
        file_path = cached_path(file_path)
        with open(file_path, "r") as text_file:
            for line in text_file:
                lang_id, sent = line.rstrip().split('\t')

                tokens = self.tokenizer.tokenize(sent)

                yield self.text_to_instance(tokens, lang_id)

In [0]:
def classify(text: str, model: LstmClassifier):
    tokenizer = CharacterTokenizer()
    token_indexers = {'tokens': SingleIdTokenIndexer()}

    tokens = tokenizer.tokenize(text)
    instance = Instance({'tokens': TextField(tokens, token_indexers)})
    logits = model.forward_on_instance(instance)['logits']
    label_id = np.argmax(logits)
    label = model.vocab.get_token_from_index(label_id, 'labels')

    print('text: {}, label: {}'.format(text, label))

In [0]:
reader = TatoebaSentenceReader()

In [7]:
train_set = reader.read('https://s3.amazonaws.com/realworldnlpbook/data/tatoeba/sentences.top10langs.train.tsv')
dev_set = reader.read('https://s3.amazonaws.com/realworldnlpbook/data/tatoeba/sentences.top10langs.dev.tsv')

0it [00:00, ?it/s]
 60%|██████    | 2667520/4414160 [00:00<00:02, 763476.15B/s]
100000it [00:09, 10486.73it/s]
0it [00:00, ?it/s]
 55%|█████▍    | 243712/443237 [00:00<00:00, 417511.09B/s]
10000it [00:01, 6148.44it/s]


In [8]:
vocab = Vocabulary.from_instances(train_set,
                                  min_count={'tokens': 3})

100%|██████████| 100000/100000 [00:01<00:00, 58096.78it/s]


In [0]:
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

In [0]:
encoder = PytorchSeq2VecWrapper(
    torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

In [0]:
model = LstmClassifier(word_embeddings, encoder, vocab, positive_label='eng')

In [0]:
iterator = BucketIterator(batch_size=32, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)

In [0]:
optimizer = optim.Adam(model.parameters())

In [0]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_set,
                  validation_dataset=dev_set,
                  num_epochs=10)

trainer.train()

In [15]:
classify('Take your raincoat in case it rains.', model)

text: Take your raincoat in case it rains., label: eng


In [20]:
classify('Tu me recuerdas a mi padre.', model)

text: Tu me recuerdas a mi padre., label: spa


In [21]:
classify('Wie organisierst du das Essen am Mittag?', model)

text: Wie organisierst du das Essen am Mittag?, label: deu


In [22]:
classify("Il est des cas où cette règle ne s'applique pas.", model)

text: Il est des cas où cette règle ne s'applique pas., label: fra


In [23]:
classify('Estou fazendo um passeio em um parque.', model)

text: Estou fazendo um passeio em um parque., label: por


In [24]:
classify('Ve, postmorgaŭ jam estas la limdato.', model)

text: Ve, postmorgaŭ jam estas la limdato., label: epo


In [25]:
classify('Credevo che sarebbe venuto.', model)

text: Credevo che sarebbe venuto., label: ita


In [27]:
classify('Nem tudja, hogy én egy macska vagyok.', model)

text: Nem tudja, hogy én egy macska vagyok., label: hun


In [28]:
classify('Nella ur nli qrib acemma deg tenwalt.', model)

text: Nella ur nli qrib acemma deg tenwalt., label: ber


In [29]:
classify('Kurşun kalemin yok, değil mi?', model)

text: Kurşun kalemin yok, değil mi?, label: tur
